In [ ]:
!pip install python_speech_features

In [ ]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
import os
import pickle
import random
import operator

In [ ]:
def distance(instance1, instance2, k):
  distance = 0
  mm1 = instance1[0]
  cm1 = instance1[1]
  mm2 = instance2[0]
  cm2 = instance2[1]
  distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
  distance+= (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
  distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
  distance-= k
  return distance



In [ ]:
def getNeighbors(trainingSet, instance, k):
  distances = []
  for x in range(len(trainingSet)):
    dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
    distances.append((trainingSet[x][2],dist))

  distances.sort(key=operator.itemgetter(1))
  neighbors = []
  for x in range(k):
    neighbors.append(distances[x][0])

  return neighbors

In [ ]:
def nearestClass(neighbors):
  classVote = {}

  for x in range(len(neighbors)):
    response = neighbors[x]
    if response in classVote:
      classVote[response] += 1
    else:
      classVote[response] = 1

  sorter = sorted(classVote.items(), key = operator.itemgetter(1),reverse=True)

  return sorter[0][0]

In [ ]:
def getAccuracy(testSet, prediction):
  correct = 0
  for x in range (len(testSet)):
    if testSet[x][-1] == predictions[x]:
      correct += 1

  return (1.0 * correct) / len(testSet)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory = '/content/drive/MyDrive/genres_original/'

In [ ]:
f = open("my.dat", 'wb')

i = 0

for folder in os.listdir(directory):
  i += 1
  if i == 11:
    break
  for file in os.listdir(directory+folder):
    try:
      (rate, sig) = wav.read(directory+folder+"/"+file)
      mfcc_feat = mfcc(sig,rate, winlen=0.020, appendEnergy=False)
      covarience = np.cov(np.matrix.transpose(mfcc_feat))
      mean_matrix = mfcc_feat.mean(0)
      feature = (mean_matrix, covarience, i)
      pickle.dump(feature, f)
    except Exception as e:
      print('Got an exception: ', e, ' in folder ', folder, ' filename: ', file)

f.close()

Got an exception:  File format b'\xcb\x15\x1e\x16'... not understood.  in folder  jazz  filename:  jazz.00054.wav


In [ ]:
dataset = []

def loadDataSet(filename, split, trSet, teSet):
  with open(filename, 'rb') as f:
    while True:
      try:
        dataset.append(pickle.load(f))
      except EOFError:
        f.close()
        break
  for x in range(len(dataset)):
    if random.random() < split:
      trSet.append(dataset[x])
    else:
      teSet.append(dataset[x])

trainingSet = []
testSet = []
loadDataSet('my.dat', 0.77, trainingSet, testSet)

In [ ]:
leng = len(testSet)
predictions = []
for x in range (leng):
  predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 15)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.5814977973568282


In [ ]:
from collections import defaultdict
results = defaultdict(int)

i = 1
for folder in os.listdir(directory):
  results[i] = folder
  i += 1

print(results)

defaultdict(<class 'int'>, {1: 'disco', 2: 'blues', 3: 'country', 4: 'hiphop', 5: 'classical', 6: 'jazz', 7: 'pop', 8: 'metal', 9: 'rock', 10: 'reggae'})


In [ ]:
test_dir = "lick.wav"

In [ ]:
rate, sig = wav.read(test_dir)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covarience = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covarience, i)

In [ ]:
pred = nearestClass(getNeighbors(dataset, feature, 5))
print(results[pred])

classical
